## Scraper

Import modules

In [73]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

import datetime
import sched, time
import sys

In [96]:
datetime.datetime.now().hour

20

Define the url to be scraped and load it

In [97]:
url='https://www.bilibili.com/'
html = urlopen(url).read().decode('utf-8')

Define a list of all sections on front page

In [98]:
secs = {'home-top':'groom-module home-card', 'home-pop':'home_popularize', 'live':'card-live-module'}

Load source to Soup

In [99]:
soup = BeautifulSoup(html, features='lxml')

In [100]:
# print(soup)

In [101]:
groom = soup.find_all('div',{'class': 'groom-module home-card'})

In [102]:
[i.a['href'].split('av')[1][:-1] for i in groom]

['22786099',
 '22791675',
 '22787517',
 '22785349',
 '22783783',
 '22761663',
 '22832807',
 '22812618']

## Write functions

In [4]:
baseurl='https://www.bilibili.com/'

def groomjob(url):
    html = urlopen(url).read().decode('utf-8')

    soup = BeautifulSoup(html, features='lxml')
    groom = soup.find_all('div',{'class': 'groom-module home-card'})

    aid = [i.a['href'].split('av')[1][:-1] for i in groom]
    return aid
    

def recordgroom():
    dt = str(datetime.datetime.now() )
    with open("data.txt", "a") as f:
        f.write( dt + ',' )
        [f.write(i+',') for i in aid[:-1]]
        f.write(aid[-1])
        f.write('\n')


hourflag = datetime.datetime.now().hour

# while True:
    
#     if datetime.datetime.now().hour > hourflag:
    
#         print('Job Running')

#         recordgroom()

#         hourflag = datetime.datetime.now().hour
    
            


## Test scheduler

In [1]:
import sched, time
s = sched.scheduler(time.time, time.sleep)

In [21]:
def print_time(a='default'):
    cur_time = time.time()
    cur_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(cur_time))
    print("From print_time", cur_time, a)

def print_some_times():
    print_time()
    s.enter(10, 1, print_time)
    s.enter(5, 2, print_time, argument=('positional',))
    s.enter(5, 1, print_time, kwargs={'a': 'keyword'})
    s.run()
    print(time.time())

# print_some_times()



In [22]:
s.empty()

False

In [13]:
dict = {}

In [78]:
def cronsched(scheduler, interval, dic, iden, action, actionargs=()):

    action(*actionargs)
    ev = scheduler.enter(interval, 1, cronsched,
                    (scheduler, interval, dic, iden, action, actionargs))
    
    dic[iden] = ev
    
def croncancel( scheduler, interval, eve ):
    
    scheduler.enter( interval, 1, scheduler.cancel, eve )

In [79]:
cronsched( s, 1, dict, 'new_vid', print_time )

From print_time 2018-05-02 19:30:05 default


In [80]:
dict

{'new_vid': Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(<sched.scheduler object at 0x112e6b390>, 1, {'new_vid': Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(...), kwargs={})}, 'new_vid', <function print_time at 0x112e6f950>, ()), kwargs={})}

In [81]:
s.queue

[Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(<sched.scheduler object at 0x112e6b390>, 1, {'new_vid': Event(time=1525311006.819536, priority=1, action=<function cronsched at 0x112f34048>, argument=(...), kwargs={})}, 'new_vid', <function print_time at 0x112e6f950>, ()), kwargs={})]

In [86]:
croncancel( s, 4, dict['new_vid'] )

In [87]:
s.empty()

False

In [92]:
s.queue

[]

In [93]:
s.run()

In [104]:
s.cancel(s.queue[0])

IndexError: list index out of range

## API

In [2]:
import json
import requests

In [82]:
api_url_base = 'https://api.bilibili.com/x/web-interface/archive/stat?aid='

In [86]:
rsp = requests.get(api_url_base + '22786099')

In [111]:
rsp.json()['data']

{'aid': 22786099,
 'coin': 27945,
 'copyright': 1,
 'danmaku': 1768,
 'favorite': 18372,
 'his_rank': 11,
 'no_reprint': 1,
 'now_rank': 0,
 'reply': 2787,
 'share': 2342,
 'view': 240198}

## Test Package

In [2]:
%%bash
ls .

bilidash
data.csv
explore.ipynb
record-front-page.py
requirements.txt
setup.py
test


In [64]:
%%bash
# pip install -e .
# pip uninstall bilidash

In [65]:
import bilidash
import sched, time
import os

In [66]:
import importlib
importlib.reload(bilidash)

<module 'bilidash' from '/Users/leima/github/uupers/bilidash/server/bilidash/__init__.py'>

In [67]:
dt_now = bilidash.current_dt_str()

In [68]:
baseurl='https://www.bilibili.com/'
html = urlopen(baseurl).read().decode('utf-8')
soup = BeautifulSoup(html, features='lxml')

In [69]:
datafile = dt_now + "-data.csv"

In [70]:
datafile

'2018-5-4-17-32-21-data.csv'

In [71]:
s = sched.scheduler(time.time, time.sleep)

# Run program
def groomjob():
    
    if os.path.exists(datafile):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not

    
    packdata = bilidash.groomaid(soup)
    bilidash.DTRecorder( bilidash.current_dt_str(), packdata, datafile, append_write)
    print('Data Dumped')

bilidash.permcronsched( s, 20, groomjob )

Data Dumped


In [49]:
bilidash.DTRecorder(dt_now, bilidash.groomjob(soup), datafile, append_write)

In [72]:
# s.cancel(s.queue[0])
s.queue

[Event(time=1525476762.724101, priority=1, action=<function permcronsched at 0x115465620>, argument=(<sched.scheduler object at 0x115e2bef0>, 20, <function groomjob at 0x1154f0ea0>, ()), kwargs={})]

In [20]:
s.run()

### Test API

In [43]:
groomdatafile = '2018-5-4-14-35-36-data.csv'
av_array = os.popen("tail -n 1 %s" % groomdatafile).read().rstrip('\n').split(',')
av_array = av_array[1:]
api_url_base = 'https://api.bilibili.com/x/web-interface/archive/stat?aid='

In [49]:
def scrape():
    
    for av in av_array:
        api_vid = av
        ret = bilidash.avapi(api_url_base+api_vid)
        avtrack_file = api_vid + '.csv'
        arr = [ ret['view'], ret['danmaku'], ret['reply'], ret['favorite'], ret['coin'], ret['share'], ret['now_rank'], ret['his_rank'], ret['no_reprint'], ret['copyright'] ]
        arr = [str(i) for i in arr ]
        
        if os.path.exists(avtrack_file):
            append_write = 'a' # append if already exists
        else:
            append_write = 'w' # make a new file if not
        
        bilidash.DTRecorder(bilidash.current_dt_str(), arr, avtrack_file, append_write)

In [50]:
scrape()

In [52]:
d = datetime.datetime.now() 

In [53]:
d

datetime.datetime(2018, 5, 4, 17, 24, 31, 149702)

In [56]:
from datetime import timedelta

In [62]:
datetime.datetime.now()  > d+timedelta(days=10)

False

In [78]:
def test():
    sys.exit

In [79]:
test()